# 🔍 Step 07: Elasticsearch 벡터 검색기 구현

이 노트북에서는 Elasticsearch를 사용하여 벡터 검색기를 구현하고 Top-K 검색을 수행합니다.

## 📝 주요 내용
1. 벡터 검색기 클래스 구현
2. 검색 파라미터 최적화
3. Top-K 검색 테스트
4. 검색 결과 평가

## 💡 벡터 검색의 이해

벡터 검색은 다음과 같은 단계로 이루어집니다:
1. 검색어를 벡터로 변환
2. 벡터 간 유사도 계산 (코사인 유사도 사용)
3. 가장 유사한 Top-K 문서 반환
4. 결과 후처리 및 정렬

## 1️⃣ 필요한 라이브러리 임포트

벡터 검색에 필요한 라이브러리들을 임포트합니다.

In [ ]:
# 기본 라이브러리 임포트
import os
import json
from typing import List, Dict, Any
from dataclasses import dataclass

# Elasticsearch 관련 라이브러리
from elasticsearch import Elasticsearch
from dotenv import load_dotenv

# OpenAI 임베딩 생성용 라이브러리
from openai import OpenAI

# 시각화 및 출력 포맷팅
from IPython.display import display, HTML
import pandas as pd

# 환경 변수 로드
load_dotenv()

# 상수 정의
INDEX_NAME = "metadata-embeddings"  # Elasticsearch 인덱스 이름

## 2️⃣ 검색 결과를 위한 데이터 클래스 정의

검색 결과를 체계적으로 관리하기 위한 데이터 구조를 정의합니다.

In [ ]:
@dataclass
class SearchResult:
    """검색 결과를 저장하는 데이터 클래스"""
    content: str          # 문서 내용
    title: str           # 문서 제목
    score: float         # 유사도 점수
    type: str           # 문서 타입
    
    def to_dict(self) -> Dict[str, Any]:
        """데이터 클래스를 딕셔너리로 변환"""
        return {
            "content": self.content,
            "title": self.title,
            "score": self.score,
            "type": self.type
        }

## 3️⃣ 벡터 검색기 클래스 구현

Elasticsearch를 사용한 벡터 검색기를 구현합니다.
- OpenAI API로 쿼리 임베딩 생성
- Elasticsearch kNN 검색 수행
- 검색 결과 후처리

In [ ]:
class VectorSearcher:
    def __init__(self):
        # OpenAI 클라이언트 초기화
        self.openai_client = OpenAI(
            api_key=os.getenv('OPENAI_API_KEY')
        )
        
        # Elasticsearch 클라이언트 초기화
        self.es_client = Elasticsearch(
            hosts=[os.getenv('ELASTICSEARCH_URL', 'http://localhost:9200')],
            basic_auth=(
                os.getenv('ELASTICSEARCH_USERNAME', 'elastic'),
                os.getenv('ELASTICSEARCH_PASSWORD', '')
            )
        )
        
        # 연결 상태 확인
        if not self.es_client.ping():
            raise ConnectionError("Elasticsearch 연결 실패")
    
    def get_query_embedding(self, query: str) -> List[float]:
        """검색어를 벡터로 변환"""
        # OpenAI API를 사용하여 쿼리 텍스트의 임베딩 생성
        response = self.openai_client.embeddings.create(
            model="text-embedding-ada-002",
            input=query
        )
        # 생성된 임베딩 벡터 반환
        return response.data[0].embedding
    
    def search(self, query: str, top_k: int = 5) -> List[SearchResult]:
        """벡터 유사도 기반 검색 수행"""
        # 쿼리 텍스트를 벡터로 변환
        query_vector = self.get_query_embedding(query)
        
        # Elasticsearch kNN 검색 쿼리 구성
        search_query = {
            "size": top_k,  # 상위 K개 결과만 반환
            "query": {
                "script_score": {
                    "query": {"match_all": {}},  # 모든 문서 대상 검색
                    "script": {
                        # 코사인 유사도 계산
                        "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                        "params": {"query_vector": query_vector}
                    }
                }
            }
        }
        
        # 검색 실행
        response = self.es_client.search(
            index=INDEX_NAME,
            body=search_query
        )
        
        # 검색 결과를 SearchResult 객체로 변환
        results = []
        for hit in response['hits']['hits']:
            source = hit['_source']
            results.append(SearchResult(
                content=source['content'],
                title=source['title'],
                score=hit['_score'],
                type=source['type']
            ))
        
        return results

## 4️⃣ 검색기 초기화 및 테스트

구현한 벡터 검색기를 초기화하고 실제 검색을 수행해봅니다.

In [ ]:
# 벡터 검색기 초기화
searcher = VectorSearcher()

# 테스트용 검색어 목록
test_queries = [
    "고객 테이블의 구조가 어떻게 되나요?",
    "주문 정보는 어떤 컬럼들을 가지고 있나요?",
    "결제 금액이 저장된 컬럼을 알려주세요"
]

# 각 검색어에 대해 검색 수행 및 결과 출력
for query in test_queries:
    print(f"\n🔍 검색어: {query}")
    print("-" * 80)
    
    # Top-5 검색 수행
    results = searcher.search(query, top_k=5)
    
    # 결과를 데이터프레임으로 변환하여 보기 좋게 출력
    df = pd.DataFrame([r.to_dict() for r in results])
    df['score'] = df['score'].round(3)  # 점수는 소수점 3자리까지만 표시
    
    # 결과 출력
    display(df[['title', 'type', 'score', 'content']])

## 5️⃣ 검색 성능 평가

검색 결과의 품질을 평가하기 위한 간단한 메트릭을 계산합니다.

In [ ]:
def evaluate_search_quality(query: str, results: List[SearchResult]) -> Dict[str, float]:
    """검색 결과의 품질을 평가하는 함수"""
    # 기본적인 메트릭 계산
    metrics = {
        # 평균 유사도 점수
        "mean_score": sum(r.score for r in results) / len(results),
        # 최고 점수
        "max_score": max(r.score for r in results),
        # 최저 점수
        "min_score": min(r.score for r in results),
        # 표준편차 (점수의 분산 정도)
        "score_std": pd.Series([r.score for r in results]).std()
    }
    
    return metrics

# 테스트 쿼리에 대한 성능 평가
print("📊 검색 성능 평가")
print("-" * 80)

for query in test_queries:
    results = searcher.search(query, top_k=5)
    metrics = evaluate_search_quality(query, results)
    
    print(f"\n검색어: {query}")
    for metric, value in metrics.items():
        print(f"- {metric}: {value:.3f}")

## 🎯 정리

이번 단계에서 완료한 작업:
1. 벡터 검색기 클래스 구현
   - OpenAI 임베딩 통합
   - Elasticsearch kNN 검색 구현
   - 검색 결과 처리

2. 검색 기능 테스트
   - 다양한 쿼리로 검색 테스트
   - 결과 시각화 및 분석
   - 성능 메트릭 계산

다음 단계에서는 이 검색기를 LangChain의 RetrievalQA 체인과 연동하여
질의응답 시스템을 구축해보겠습니다.